图像有几种不同的文件格式，但幸运的是，在 Python 中有很多加载图像的方法。让我们从使用 imageio 模块加载 PNG 格式图像开始

In [1]:
import numpy as np
import torch
# 设置打印选项
"""  
edgeitems: 这个参数决定了当一个张量过大，无法全部打印时，每个维度打印多少个元素。在这个例子中，edgeitems=2 表示每个维度只打印前两个和后两个元素。
threshold: 这个参数决定了张量的大小超过多少时，才使用省略的方式打印。
            在这个例子中，threshold=50 表示当张量的元素数量超过50时，就不会全部打印出来，而是使用省略的方式。
"""
torch.set_printoptions(edgeitems=2, threshold=50)

In [5]:
import imageio

img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

C:\Users\12279\AppData\Local\Temp\ipykernel_23216\1529188539.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')


(720, 1280, 3)

此时， img 是一个具有 3 个维度的类 NumPy 数组对象： 2 个空间维度尺寸—宽度和高度，
第 3 个维度对应红色、绿色和蓝色通道。任何输出 NumPy 数组的库都可以获得一个 PyTorch 张量，唯一需要注意的是维度布局。处理图像数据的 PyTorch 模块要求张量排列为 C×H×W（分别表示通道、高度和宽度）

我们可以用张量的 permute()方法使每个新的维度利用旧维度得到一个合适的布局。给定一个已知的 H×W×C 的输入张量，我们先布局通道 2，然后是通道 0 和通道 1，从而得到一个合适的布局:

In [7]:
img = torch.from_numpy(img_arr)
out = img.permute(2,0,1)
img.shape, out.shape

(torch.Size([720, 1280, 3]), torch.Size([3, 720, 1280]))

按照我们对早期数据类型使用的策略，创建一个多图像的数据集作为神经网络的输入，我们沿着第一维批量存储图像，以获得一个 N × C × H × W 的张量。

与使用 stack()构建张量相比，一个更有效的替代方法是，我们可以预先分配一个适当大小的张量，并使用从目录中加载的图像填充它，像这样：

In [9]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
batch.shape

torch.Size([3, 3, 256, 256])

这表明我们的批处理将由 3 幅高度 256 像素、宽度 256 像素的 RGB 图像组成。请注意张量的类型： 我们期望每种颜色都以 8 位整数表示， 就像标准消费级数码相机的大多数摄影格式一样。我们现在可以从一个输入目录中加载所有的 PNG 图像，并将它们存储在张量中：

In [16]:
import os
data_dir = '../data/p1ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir)
            if os.path.splitext(name)[-1] == '.png']
filenames

['cat1.png', 'cat2.png', 'cat3.png']

In [17]:
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr).permute(2,0,1)[:3] # 这里我们只保留前 3 个通道，有时图像还有一个表示透明度的 alpha 通道，但我们的网络只需要 RGB 输入
    batch[i] = img_t

C:\Users\12279\AppData\Local\Temp\ipykernel_23216\2359010066.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


我们之前提到过， 神经网络通常使用浮点数张量作为输入。当输入数据的范围为 0～1 或−1～1 时，神经网络表现出最佳的训练性能， 这是其构建块的定义方式所决定的。

我们要做的一件典型的事情就是将张量转换为浮点数并对像素的值进行归一化。 将张量强制转换为浮点数很容易，但是归一化就比较棘手了，因为这取决于输入范围（ 0～1 或−1～1）。正规化的一种方法是将像素值除以 255（ 8 位无符号二进制数可表示的最大数字）：

In [19]:
batch = batch.float() / 255.0

正规化的另一种方法是计算输入数据的均值和标准差， 并对其进行缩放，使每个通道的均值为 0，标准差为 1：

In [20]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std